# 本筆記讓大家有概念，如何使用pretrained model來做圖像分類。

以下程式碼大多參考自Keras作者François Chollet的 [**教學**](https://blog.keras.io/building-powerful-image-classification-models-using-very-little-data.html) 以及他的書：[deep-learning-with-python](https://www.manning.com/books/deep-learning-with-python)。於新版Keras(```v2.1.2```)下，他的範例無法執行，故我有稍做修正。

# 筆記內容：

* [略為了解一下資料集](#01)
* [利用pretrained model建置模型](#02)
* [模型微調](#03)

In [ ]:
# # =====================================================================
# # 由於課堂上可能有多人共用同一顆GPU，以下限定使用者只能用計算卡上面一半的記憶體。
# import tensorflow as tf
# from keras.backend.tensorflow_backend import set_session
# config = tf.ConfigProto()
# config.gpu_options.per_process_gpu_memory_fraction = 0.5 # 使用一半記憶體
# set_session(tf.Session(config=config))
# # =====================================================================

In [ ]:
import matplotlib.pyplot as plt
%matplotlib inline
import cv2
import numpy as np

In [ ]:
from keras import applications
from keras.preprocessing.image import ImageDataGenerator
from keras import optimizers
from keras.models import Sequential
from keras.layers import Dropout, Flatten, Dense
from keras.models import load_model
from keras.models import Model

## <a id='01'>略為了解一下資料集</a>

In [ ]:
num_train_dogs,=!ls ../datasets/cats_and_dogs/train/dogs | wc -l
num_train_cats,=!ls ../datasets/cats_and_dogs/train/cats | wc -l
num_test_dogs,=!ls ../datasets/cats_and_dogs/validation/dogs | wc -l
num_test_cats,=!ls ../datasets/cats_and_dogs/validation/cats | wc -l

print("狗的train/test張數=",num_train_dogs,num_test_dogs)
print("貓的train/test張數=",num_train_cats,num_test_cats)

看一張貓的照片:

In [ ]:
# 隨機載入一張貓的圖片
fig_idx=np.random.choice(1000)
img=cv2.imread("../datasets/cats_and_dogs/train/cats/cat.%s.jpg"%fig_idx)
# 繪圖
plt.imshow(img)

看一張狗的照片:

In [ ]:
# 隨機載入一張狗的圖片
fig_idx=np.random.choice(1000)
img=cv2.imread("../datasets/cats_and_dogs/train/dogs/dog.%s.jpg"%fig_idx)
# 繪圖
plt.imshow(img)

[回索引](#%E7%AD%86%E8%A8%98%E5%85%A7%E5%AE%B9%EF%BC%9A)

## 訓練模型

In [ ]:
!ls -hl ../pretrain_models/

In [ ]:
# 紀錄兩個放置模型參數的檔案路徑
path_with_top="../pretrain_models/vgg16_weights_tf_dim_ordering_tf_kernels.h5"
path_without_top="../pretrain_models/vgg16_weights_tf_dim_ordering_tf_kernels_notop.h5"

In [ ]:
# #看一下模型架構(without top)
# model = applications.VGG16(weights=path_without_top, include_top=False)
# model.summary()

In [ ]:
# #看一下模型架構(with top)
# model = applications.VGG16(weights=path_with_top, include_top=True)
# model.summary()

### <a id='02'>利用pretrained model建置模型</a>

我們即將建立的模型可分為兩個部分：pretrained convolutional part以及final classification part。

其中，pretrained convolutional part是由已經訓練好的VGG16模型擷取而出。這個已訓練好的VGG16模型是由ImageNet資料集訓練而來。

而final classification part並無做過任何訓練，也就是說，它裡面的參數是被隨機初始化的。

我們將pretrained convolutional part結合後面的final classification part之後，丟入貓狗資料集，來訓練final classification part內的權重參數。

於訓練過程中，我們不會訓練pretrained convolutional part，這是因為它已經內含一些"規則"，能夠將原本的圖片整理成適合拿來分類貓狗的"高階特徵"。有了這些適合分類貓狗的高階特徵後，final classification part就可以比較容易的利用這些高階特徵去做貓狗分類了。

In [ ]:
train_data_dir = '../datasets/cats_and_dogs/train/'           # 訓練資料位置
validation_data_dir = '../datasets/cats_and_dogs/validation/' # 驗證資料位置


# 建立VGG 16網路(前面conv block的部分)
conv_base = applications.VGG16(weights=path_without_top,
                               include_top=False,
                               input_shape=(150,150,3))
conv_base.summary()

In [ ]:
train_data_dir = '../datasets/cats_and_dogs/train/'           # 訓練資料位置
validation_data_dir = '../datasets/cats_and_dogs/validation/' # 驗證資料位置


# 建立VGG 16網路(前面conv block的部分)
conv_base = applications.VGG16(weights=path_without_top,
                               include_top=False,
                               input_shape=(150,150,3))
# 於conv block之後附加一個用於分類的block
x = Flatten()(conv_base.output)
x = Dense(256, activation='relu')(x)
x = Dropout(0.5)(x)
preds = Dense(1, activation='sigmoid')(x)
# 將conv block和末端用於分類的block結合成一個模型
model = Model(conv_base.input, preds)
# 告知模型，不需要訓練前面的conv layers (我們只要訓練後面用於分類的block)
conv_base.trainable = False
# 編譯模型，告知模型訓練方式
model.compile(loss='binary_crossentropy',
              optimizer=optimizers.RMSprop(lr=2e-5),
              metrics=['acc'])
# ============================================================
# 產生train/test generator。generator會從資料夾內撈出一個批次的圖，
# 並將該批次的圖像做augmentation。
train_datagen = ImageDataGenerator(
      rescale=1./255,
      rotation_range=40,
      width_shift_range=0.2,
      height_shift_range=0.2,
      shear_range=0.2,
      zoom_range=0.2,
      horizontal_flip=True,
      fill_mode='nearest')

test_datagen = ImageDataGenerator(rescale=1./255) 

train_generator = train_datagen.flow_from_directory(
        train_data_dir,                                       
        target_size=(150, 150),                          
        batch_size=20,
        class_mode='binary')                             

validation_generator = test_datagen.flow_from_directory(
        validation_data_dir,
        target_size=(150, 150),
        batch_size=20,
        class_mode='binary')
# ============================================================

# 訓練模型。圖片將以generator的方式餵入模型。
history = model.fit_generator(
      train_generator,
      steps_per_epoch=100,
      epochs=15,
      validation_data=validation_generator,
      validation_steps=50)
#畫出訓練過程
plt.plot(history.history['acc'],ms=5,marker='o',label='accuracy')
plt.plot(history.history['val_acc'],ms=5,marker='o',label='val accuracy')
plt.legend()
plt.show()

[回索引](#%E7%AD%86%E8%A8%98%E5%85%A7%E5%AE%B9%EF%BC%9A)

練習：試著自建一個簡單的CNN模型(不超過5個Convolutional layer)，看你能達成多高的模型準確率。

In [ ]:
# # 練習於此
# model = models.Sequential()
# model.add(layers.Conv2D(32, (3, 3), activation='relu',
#                         input_shape=(150, 150, 3)))
# model.add(...)
# ...

[回索引](#%E7%AD%86%E8%A8%98%E5%85%A7%E5%AE%B9%EF%BC%9A)

### <a id='03'>模型微調</a>

先看一下模型架構：

In [ ]:
model.summary()

先前有提到，我們不建議將整個convolutional part拿去做訓練。事實上，這是因為該part的最前面幾個layers內含一些常見規則，可以把原圖片轉換成一些基本特徵。又，因為那些基本特徵是相當適用於各種影像分類場合的，所以我們不需要將那些layers裡面的權重參數再去做重新訓練。

不過，訓練部分convolutional part則是合理的。除了訓練classification part之外，我們還可以選擇針對convolutional part的最後幾個layers來做再訓練。這是因為，那些後面的layers可能內含一些規則，可以把原圖片轉換成較複雜的高階特徵。然而，事實上，這些後面的layers所轉換出來的高階特徵是比較針對於ImageNet圖片集的分類。換句話說，那些高階特徵是被整理出來，專門用來分類1000類各種形形色色的物體的(ImageNet內含1000類圖片)。因此，最後幾個layer所內含的規則，有可能會過於複雜，有可能會不完全適用於較容易的貓狗圖片分類。

因此，以下我們建模的時候，除了訓練後面的classification part之外，我們還多訓練了convolutional part的最後一個block (block 5)。

In [ ]:
# 將conv block5調整為可訓練
conv_base.trainable = True

set_trainable = False
for layer in conv_base.layers:
    if layer.name == 'block5_conv1':
        set_trainable = True
    if set_trainable:
        layer.trainable = True
    else:
        layer.trainable = False

In [ ]:
# 編譯模型，告知模型訓練方式
model.compile(loss='binary_crossentropy',
              optimizer=optimizers.RMSprop(lr=1e-5),
              metrics=['acc'])
# 訓練模型
history = model.fit_generator(
      train_generator,
      steps_per_epoch=100,
      epochs=15,
      validation_data=validation_generator,
      validation_steps=50)
# 畫出訓練過程
plt.plot(history.history['acc'],ms=5,marker='o',label='accuracy')
plt.plot(history.history['val_acc'],ms=5,marker='o',label='val accuracy')
plt.legend()
plt.show()

[回索引](#%E7%AD%86%E8%A8%98%E5%85%A7%E5%AE%B9%EF%BC%9A)